Let $\Sigma_t$ denote portfolio of the instruments under our managment
$$\Sigma_t = n_i(t) \cdot I_i(t),$$
where $n_i(t)$ -- position in the instrument $i$, $I_i(t)$ -- its present value

$$\dot\Sigma(t) = \dot n_i I_i + n_i \dot I_i$$

So we must implement Instruments' dynamics and portolio strategy to control $n_i$ 

In [1]:
import numpy as np
import pandas as pd
from LPtools.utils import Adapter, Runner
from LPtools.clients import Client
from LPtools.instruments import UniPool, Bond, Portfolio, Position, Cash, Perpetual
from LPtools.instruments import Holder, PerpetHedger
from LPtools.solver import Solver

In [2]:
zcurve = pd.read_csv('zcyc_rates.csv', index_col=0, parse_dates=True, infer_datetime_format=True, na_filter=False)

adapter = Adapter(pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                  symbol="ETHUSDT",
                  zcurve=zcurve)

#response = adapter(300)

In [3]:
#response.to_csv('saved_response.csv')

In [4]:
response = pd.read_csv('saved_response.csv', index_col=0, infer_datetime_format=True)
init_state = response.iloc[0]

In [6]:
solver = Solver(response)
results = solver.solve(t = 20,
             delta1=.05, q1=.98,
             delta2=.1, q2=.98,
             verbose=True
            )
pl, pu = map(lambda x: x*init_state.token0Price, results.x)

1. Hodl

In [7]:
perpetual = Position(tag='perpet', instrument=Perpetual())
bond = Position(tag='bond', instrument=Bond(init_state))

positions = [perpetual, bond]
balancer = Holder(positions, initial_amounts=[1 / init_state.token0Price, -1])
portfolio_hodl = Portfolio(balancer, positions=positions)

2. Passive pool

In [8]:

pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=pl, price_upper=pu, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))


positions = [pool, bond]
balancer = Holder(positions, initial_amounts=[1,-1])
portfolio_pooler = Portfolio(balancer, positions=positions)

3. Hedging

In [9]:
pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=pl, price_upper=pu, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))
perpetual = Position(tag='perpet', instrument=Perpetual())

positions = [pool, perpetual, bond]

delta =  pool.instrument.L / init_state.token0Price**.5
balancer = PerpetHedger(
    positions,
    initial_amounts=[1, - delta, delta * init_state.token0Price - 1 ],
    rebalancing_interval= 2
)
portfolio_perpet = Portfolio(balancer, positions)

In [10]:
runner = Runner(response,[portfolio_hodl, portfolio_pooler, portfolio_perpet])
runner.run()

In [11]:
runner.summary

,total_value,discounted_value
portfolio 0,0.285268,0.274328
portfolio 1,0.387027,0.367990
portfolio 2,0.420783,0.397622


In [12]:
runner.logs(2)

,pool_value,pool_amount,perpet_value,perpet_amount,bond_value,bond_amount,payments,transaction_costs
0,1.000013,0,3496.292879,0.000000,1.000000,0.0,0.000000e+00,-1.500143e-02
1,0.998211,1,3483.701459,-0.000143,1.000139,-0.5,-1.406753e-06,-2.563793e-09
2,1.054052,1,3884.370431,-0.000143,1.000279,-0.5,8.526746e-05,0.000000e+00
3,1.057450,1,3909.453752,-0.000143,1.000418,-0.5,-9.652607e-07,-8.026025e-08
4,1.065387,1,3968.366567,-0.000135,1.000558,-0.5,1.167594e-05,0.000000e+00
...,...,...,...,...,...,...,...,...
204,1.075175,1,4041.612472,-0.000126,1.028853,-0.5,-3.851266e-05,0.000000e+00
205,1.082566,1,4097.371402,-0.000126,1.028997,-0.5,2.093576e-06,-6.367797e-08
206,1.108335,1,4294.761309,-0.000132,1.029140,-0.5,7.882936e-06,0.000000e+00
207,1.127702,1,4446.166582,-0.000132,1.029284,-0.5,5.927545e-06,-5.287535e-08
